# Correlation Analysis Talapas TTbar70

Author: Brain Gravelle (gravelle@cs.uoregon.edu)


All this is using the taucmdr python libraries from paratools
http://taucommander.paratools.com/

<a href='#top'>top</a><br>

## Imports
This section imports necessary libraies, the metrics.py and utilities.py files and sets up the window.


<a id='top'></a>

In [1]:
# A couple of scripts to set the environent and import data from a .tau set of results
from utilities import *
from metrics import *
# Plotting, notebook settings:
%matplotlib inline  
#plt.rcParams.update({'font.size': 16})
import numbers
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.float_format', lambda x: '%.2e' % x)
pd.set_option('display.max_columns',100)
pd.set_option('max_colwidth', 70)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import copy

## Getting Data
<a id='data'></a>

TAU Commander uses TAU to run the application and measure it using runtime sampling techniques (similar to Intel VTune). Many customization options are available. For example, we may consider each function regardless of calling context, or we may decide to enable callpath profiling to see each context separately.

From the talapas_scaling application the following experiments are available. These use Talapas (with 28 thread Broadwell processors) and the build-ce (realistic) option for mkFit. The first six experiments use the --num-thr option to set the thread count which is intended to perform threading within the events. the last two add the --num-ev-thr option to set the event threads, so that all threads are used to process events in parallel and each event is processed by a single thread. 
* manual_scaling_Large_talapas		
* manual_scaling_Large_talapas_fullnode	
* manual_scaling_TTbar70_talapas		
* manual_scaling_TTbar70_talapas_fullnode
* manual_scaling_TTbar35_talapas
* manual_scaling_TTbar35_talapas_fullnode
* ev_thr_scaling_Large_talapas
* ev_thr_scaling_Large_talapas_fullnode

Additionally available in the cori_scaling application are the following. These were run on NERSC's Cori on the KNL with the default memory settings (quad - 1 NUMA domain, cache - MCDRAM as direct mapped cache). See http://www.nersc.gov/users/computational-systems/cori/running-jobs/advanced-running-jobs-options/ for more info on the KNL modes. Similar to the talapas scaling they use the build-ce option and threading within each event.
* manual_scaling_TTbar35
* manual_scaling_TTbar70
* manual_scaling_Large
* mixed_thr_scaling_Large - this is bad


### Importing Scaling Data - Talapas TTbar70 is current
Here we import the data. In this case we are using Cori data from the experiments with the threads working within each event using the TTbar70 file. Note that this box will take an hour or more to run; please go enjoy a coffee while you wait.

In [2]:
application = "talapas_scaling"
experiment  = "manual_scaling_TTbar70_talapas_fullnode"
# experiment  = "manual_scaling_Large_talapas"
# experiment = "ev_thr_scaling_Large_talapas"

# application = "cori_scaling"
# experiment  = "manual_scaling_TTbar35"
# experiment  = "manual_scaling_TTbar70"
# experiment  = "manual_scaling_Large"
# experiment  = "mixed_thr_scaling_Large"

path = ".tau/" + application + "/" + experiment + "/"
# note that this function takes a long time to run, so only rerun if you must

metric_data = get_pandas_scaling(path, callpaths=True)
    
if application == "talapas_scaling":
    metric_data = remove_erroneous_threads(metric_data,  [1, 8, 16, 32, 48, 56])
elif application == "cori_scaling":
    print(metric_data.keys())
    metric_data = remove_erroneous_threads(metric_data,  [1, 4, 8, 16, 32, 48, 64, 80, 96, 112, 128, 144, 160, 176, 192, 208, 224, 240, 256])

Found: 900 trials with 166 errors




## Adding metrics
<a id='metrics'></a>

metrics are available in metrics.py. At this time the following can be added:
* add_IPC(metrics)          - Instructions per Cycle
* add_CPI(metrics)          - Cycles per instruction
* add_VIPC(metrics)         - vector instructions per cycle
* add_VIPI(metrics)         - vector instructions per instruction (i.e. fraction of total)
* add_L1_missrate(metrics)  - miss rate for L1 cache

for scaling data please use the add_metric_to_scaling_data(data, metric_func) function to add a metric

Here we add some predeefined metrics and print the top 10 functions with the best IPC

In [3]:
add_metric_to_scaling_data(metric_data, add_CPI)
add_metric_to_scaling_data(metric_data, add_IPC)
add_metric_to_scaling_data(metric_data, add_L1_missrate)
add_metric_to_scaling_data(metric_data, add_L2_missrate)

if application == 'cori_scaling': 
    llc = True
    add_metric_to_scaling_data(metric_data, add_VIPI)
else: 
    llc = False
#     add_metric_to_scaling_data(metric_data, add_DERIVED_SP_VOPO)
    
add_metric_to_scaling_data(metric_data, add_L3_missrate, llc)
add_metric_to_scaling_data(metric_data, add_DERIVED_BRANCH_MR)

print_available_metrics(metric_data, scaling=True)
print(metric_data.keys())
# To test
# metric_data[32]['DERIVED_SP_VOPO'].sort_values(by='Exclusive',ascending=False).head(10)

PAPI_L2_TCA
PAPI_TLB_DM
PAPI_VEC_DP
PAPI_L2_TCM
DERIVED_CPI
PAPI_NATIVE_FP_ARITH:256B_PACKED_SINGLE
PAPI_L1_TCM
PAPI_BR_MSP
PAPI_NATIVE_MACHINE_CLEARS:COUNT
PAPI_DP_OPS
PAPI_RES_STL
PAPI_TOT_INS
PAPI_VEC_SP
PAPI_BR_CN
PAPI_L3_TCA
PAPI_TLB_IM
PAPI_NATIVE_FP_ARITH:128B_PACKED_SINGLE
DERIVED_L1_MISSRATE
PAPI_L3_TCM
DERIVED_L3_MISSRATE
PAPI_NATIVE_FP_ARITH:SCALAR_DOUBLE
PAPI_NATIVE_RESOURCE_STALLS:SB
PAPI_NATIVE_FP_ARITH:SCALAR_SINGLE
PAPI_BR_INS
DERIVED_L2_MISSRATE
PAPI_NATIVE_FP_ARITH:128B_PACKED_DOUBLE
DERIVED_BRANCH_MR
DERIVED_IPC
PAPI_LST_INS
PAPI_NATIVE_UOPS_RETIRED:STALL_CYCLES
PAPI_SP_OPS
PAPI_NATIVE_FP_ARITH:256B_PACKED_DOUBLE
PAPI_TOT_CYC
[32, 48, 56, 8, 16]


#### Combining metrics

In [9]:
thr_li = metric_data.keys()
alldata = {}
for thr in thr_li:
    alldata[thr] = combine_metrics(metric_data[thr],inc_exc='Exclusive')

## Correlations
<a id='corr'></a>

In [19]:
THREAD_CNT = 8

metric_li = ['PAPI_TOT_CYC','PAPI_RES_STL']

corrs = [alldata[THREAD_CNT].corr('pearson').fillna(0)[metric_li], alldata[THREAD_CNT].corr('kendall').fillna(0)[metric_li], alldata[THREAD_CNT].corr('spearman').fillna(0)[metric_li]]

corr_data = corrs[0][['PAPI_TOT_CYC']].copy()
corr_data['p_' + metric_li[0]] = corr_data[metric_li[0]]
corr_data['p_' + metric_li[1]] = corrs[0][metric_li[1]]

corr_data['k_' + metric_li[0]] = corrs[1][metric_li[0]]
corr_data['k_' + metric_li[1]] = corrs[1][metric_li[1]]

corr_data['s_' + metric_li[0]] = corrs[2][metric_li[0]]
corr_data['s_' + metric_li[1]] = corrs[2][metric_li[1]]

corr_data.drop(['PAPI_TOT_CYC'],axis=1,inplace=True)

corr_data.style.format("{:.2%}").background_gradient(cmap=cm)

,p_PAPI_TOT_CYC,p_PAPI_RES_STL,k_PAPI_TOT_CYC,k_PAPI_RES_STL,s_PAPI_TOT_CYC,s_PAPI_RES_STL
PAPI_TOT_CYC,100.00%,99.76%,100.00%,96.48%,100.00%,99.77%
PAPI_L2_TCA,99.80%,99.80%,96.04%,96.13%,99.71%,99.72%
PAPI_TLB_DM,99.43%,99.80%,93.89%,94.45%,99.35%,99.45%
PAPI_VEC_DP,54.84%,57.89%,91.11%,90.60%,98.78%,98.67%
PAPI_L2_TCM,99.62%,99.58%,95.20%,95.91%,99.60%,99.71%
DERIVED_CPI,-2.99%,-2.98%,-7.91%,-10.47%,-12.03%,-15.53%
PAPI_NATIVE_FP_ARITH:256B_PACKED_SINGLE,97.55%,96.69%,71.66%,73.00%,75.85%,77.43%
PAPI_L1_TCM,99.58%,99.35%,96.80%,96.61%,99.82%,99.79%
PAPI_BR_MSP,99.74%,99.75%,94.18%,93.25%,99.42%,99.24%
PAPI_NATIVE_MACHINE_CLEARS:COUNT,99.13%,99.66%,94.23%,94.56%,99.43%,99.48%


## Correlations
<a id='corr'></a>

In [15]:
THREAD_CNT = 32

cm = sns.light_palette("yellow", as_cmap=True)
    
pcorr = get_corr(alldata[THREAD_CNT])
kcorr = get_corr(alldata[THREAD_CNT], method='kendall')
scorr = get_corr(alldata[THREAD_CNT], method='spearman')

TypeError: get_corr() takes at least 2 arguments (1 given)

In [16]:
metric_li = ['PAPI_TOT_CYC','PAPI_RES_STL']

corrs = [alldata.corr('pearson').fillna(0)[metric_li], alldata.corr('kendall').fillna(0)[metric_li], alldata.corr('spearman').fillna(0)[metric_li]]

corr_data = corrs[0][['PAPI_TOT_CYC']].copy()
corr_data['p_' + metric_li[0]] = corr_data[metric_li[0]]
corr_data['p_' + metric_li[1]] = corrs[0][metric_li[1]]

corr_data['k_' + metric_li[0]] = corrs[1][metric_li[0]]
corr_data['k_' + metric_li[1]] = corrs[1][metric_li[1]]

corr_data['s_' + metric_li[0]] = corrs[2][metric_li[0]]
corr_data['s_' + metric_li[1]] = corrs[2][metric_li[1]]

corr_data.drop(['PAPI_TOT_CYC'],axis=1,inplace=True)

corr_data.style.format("{:.2%}").background_gradient(cmap=cm)

AttributeError: 'dict' object has no attribute 'corr'

## Correlations
<a id='corr'></a>

In [17]:
THREAD_CNT = 56

cm = sns.light_palette("yellow", as_cmap=True)
    
pcorr = get_corr(alldata[THREAD_CNT])
kcorr = get_corr(alldata[THREAD_CNT], method='kendall')
scorr = get_corr(alldata[THREAD_CNT], method='spearman')

TypeError: get_corr() takes at least 2 arguments (1 given)

In [18]:
metric_li = ['PAPI_TOT_CYC','PAPI_RES_STL']

corrs = [alldata.corr('pearson').fillna(0)[metric_li], alldata.corr('kendall').fillna(0)[metric_li], alldata.corr('spearman').fillna(0)[metric_li]]

corr_data = corrs[0][['PAPI_TOT_CYC']].copy()
corr_data['p_' + metric_li[0]] = corr_data[metric_li[0]]
corr_data['p_' + metric_li[1]] = corrs[0][metric_li[1]]

corr_data['k_' + metric_li[0]] = corrs[1][metric_li[0]]
corr_data['k_' + metric_li[1]] = corrs[1][metric_li[1]]

corr_data['s_' + metric_li[0]] = corrs[2][metric_li[0]]
corr_data['s_' + metric_li[1]] = corrs[2][metric_li[1]]

corr_data.drop(['PAPI_TOT_CYC'],axis=1,inplace=True)

corr_data.style.format("{:.2%}").background_gradient(cmap=cm)

AttributeError: 'dict' object has no attribute 'corr'